采用 HIVE_CODE V2 的集成机器学习方法来对时间序列进行分类


In [1]:
import pandas as pd
import warnings
from sklearn.metrics import classification_report
warnings.filterwarnings('ignore')
import numpy as np
np.seterr(divide='ignore',invalid='ignore')
import pickle as pkl

1.加载数据


In [8]:
def load_file_with_p(file_path):
    '''载入,p二进制文件'''
    return pkl.load(open(file_path,'rb'))

def save_file_with_p(save_df,file_path):
    '''载入,p二进制文件'''
    pkl.dump(save_df,open(file_path,'wb'))

In [9]:
train_X = np.load('/train_x.npy') 
train_y = np.load('/train_y.npy') 

test_X = np.load('/test_x_A.npy')

train_X=train_X[:15000]
train_y=train_y[:15000]
print(f'训练集：{train_X.shape} 标签：{train_y.shape}')
print(f'测试集{test_X.shape}')

训练集：(15000, 2, 180) 标签：(15000,)
测试集(1155, 2, 180)


2. 五折交叉验证训练模型


In [10]:
train_X = train_X.astype(np.int32)
test_X = test_X.astype(np.int32)
train_y = train_y.astype(np.int32)

In [12]:
from sklearn.model_selection import KFold
from aeon.classification.hybrid import HIVECOTEV2
from aeon.classification.convolution_based import MultiRocketHydraClassifier

results = []
kf = KFold(n_splits=5, shuffle=False)
cur_fold = 0
for train_index, valid_index in kf.split(train_X): 
    print(f'The Fold {cur_fold} is training')
    # model = MultiRocketHydraClassifier(n_groups=32,random_state=0)  
    model = HIVECOTEV2()

    X_train, x_valid = train_X[train_index], train_X[valid_index] 
    y_train, y_valid = train_y[train_index], train_y[valid_index] 

    model.fit(X_train, y_train) # 训练
    y_predict_rate = model.predict(x_valid) # 预测
    result = classification_report(y_valid, y_predict_rate, output_dict=True)# 多分类测试结果  ,[p, r, best_f1_score]
    print(result)
    results.append(result)
    save_file_with_p(model,f'/home/gsl/workspace/jd/model/H2/fold_{cur_fold}.p')
    cur_fold+=1

The Fold 0 is training


In [17]:
from aeon.classification.convolution_based import MultiRocketHydraClassifier
from aeon.testing.utils.data_gen import make_example_3d_numpy
X, y = make_example_3d_numpy(n_cases=10, n_channels=1, n_timepoints=12,
                             random_state=0)
clf = MultiRocketHydraClassifier(random_state=0)  
# clf.fit(X, y)  

# clf.predict(X)  
X.dtype

dtype('float64')

In [3]:
from sklearn.model_selection import KFold
from aeon.classification.hybrid import HIVECOTEV2
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime
from aeon.classification.convolution_based import MultiRocketHydraClassifier

def train_and_evaluate(train_index, valid_index):
    # model = HIVECOTEV2()
    model = MultiRocketHydraClassifier(random_state=0)  
    print(train_index)
    X_train, x_valid = train_X[train_index], train_X[valid_index] 
    y_train, y_valid = train_y[train_index], train_y[valid_index] 

    model.fit(X_train, y_train)
    y_predict_rate = model.predict(x_valid) # 预测
    result = classification_report(y_valid, y_predict_rate, output_dict=True)# 多分类测试结果  ,[p, r, best_f1_score]
    print(result)
    current_time = datetime.now()

    save_file_with_p(model,f'/H2/fold_{current_time}.p')


kf = KFold(n_splits=5, shuffle=True)
with ThreadPoolExecutor(max_workers=5) as executor:
    futures = []

    for train_index, test_index in kf.split(train_X):
        futures.append(executor.submit(train_and_evaluate, train_index, test_index))

[   0    1    2 ... 9997 9998 9999]
[   3    5    6 ... 9997 9998 9999]
[   0    1    2 ... 9997 9998 9999]
[   0    1    2 ... 9996 9998 9999]
[   0    1    2 ... 9993 9994 9997]


In [ ]:
clf = MultiRocketHydraClassifier(random_state=0)  